In [8]:
from mc_moea import *

In [46]:
import imageio

In [9]:
import pickle
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [10]:
fname_l = glob.glob("solutions/*.pkl.bz2")

In [16]:
solutions = pd.read_pickle("solutions/best_of_iterations.pkl.bz2")

In [17]:
solutions

,solution_id,solution_obj,from_generation,signal_strength,construction_cost,num_orphans,fitness_value
0,15e86c49-3ffd-4f63-8c8e-5e27e137db42,Solution : 15e86c49-3ffd-4f63-8c8e-5e27e137db4...,-1,-103.611215,10650000,0,1.208170
494,520f00fd-585f-41e9-85a4-95559cc8a888,Solution : 520f00fd-585f-41e9-85a4-95559cc8a88...,1,-99.736935,11820000,0,1.202199
414,4333c694-75e7-4432-8954-01b53a5f6b76,Solution : 4333c694-75e7-4432-8954-01b53a5f6b7...,2,-102.907313,11310000,0,1.218271
478,d0d90393-093d-4bed-8bb7-e53c0c627a5e,Solution : d0d90393-093d-4bed-8bb7-e53c0c627a5...,3,-101.031475,11120000,0,1.196468
498,e20b9303-f646-44b8-8ebb-43a4ba0e0a27,Solution : e20b9303-f646-44b8-8ebb-43a4ba0e0a2...,4,-102.582403,11300000,0,1.215067
...,...,...,...,...,...,...,...
459,7b738c41-ef5e-4615-a6f1-f6fab3c1c623,Solution : 7b738c41-ef5e-4615-a6f1-f6fab3c1c62...,95,-99.481925,12280000,0,1.211381
479,f26cb537-212d-4f0d-ad35-6e7024a3d9d3,Solution : f26cb537-212d-4f0d-ad35-6e7024a3d9d...,96,-100.829262,12330000,0,1.224880
438,65a7044b-64fc-4af7-87f9-a85cf5ad901e,Solution : 65a7044b-64fc-4af7-87f9-a85cf5ad901...,97,-98.023692,12570000,0,1.205374
416,a61c9b77-e22a-419e-8998-76456bdcd38a,Solution : a61c9b77-e22a-419e-8998-76456bdcd38...,98,-103.747431,10970000,0,1.217408


In [40]:
pos = np.array([[s.x, s.y] for s in solutions["solution_obj"].iloc[0].stations])

---

In [18]:
data = []
fnames = glob.glob("position/*.dat")
for fname in fnames:
    X = pickle.load(open(fname, "rb"))
    data.append(X)

In [19]:
bound_max = 800
bound_min = -800

In [48]:
for i in tqdm(range(solutions.shape[0])):
    pos = np.array([[s.x, s.y] for s in solutions["solution_obj"].iloc[i].stations])
    fig, axs = plt.subplots(2,3, figsize=(17,10), sharex=True, sharey=True)
    fig.suptitle(f'Solutions from Iter: {i:04d}', fontsize=16)
    plt.xlim(-900, 900)
    plt.ylim(-900, 900)

    axs[0][0].scatter(data[0][:,0], data[0][:,1], s=1.5)
    axs[0][0].scatter(pos[:,0], pos[:,1], s=30)
    axs[0][0].set_title("Pattern I (P = 0.32)")

    axs[0][1].scatter(data[1][:,0], data[1][:,1], s=1.5)
    axs[0][1].scatter(pos[:,0], pos[:,1], s=30)
    axs[0][1].set_title("Pattern II (P = 0.25)")

    axs[0][2].scatter(data[2][:,0], data[2][:,1], s=1.5)
    axs[0][2].scatter(pos[:,0], pos[:,1], s=30)
    axs[0][2].set_title("Pattern III (P = 0.17)")

    axs[1][0].scatter(data[3][:,0], data[3][:,1], s=1.5)
    axs[1][0].scatter(pos[:,0], pos[:,1], s=30)
    axs[1][0].set_title("Pattern IV (P = 0.14)")

    axs[1][1].scatter(data[4][:,0], data[4][:,1], s=1.5)
    axs[1][1].scatter(pos[:,0], pos[:,1], s=30)
    axs[1][1].set_title("Pattern V (P = 0.09)")

    axs[1][2].scatter(data[5][:,0], data[5][:,1], s=1.5)
    axs[1][2].scatter(pos[:,0], pos[:,1], s=30)
    axs[1][2].set_title("Pattern VI (P = 0.03)")

    plt.savefig(f"figure/output-{i:04d}.png")
    plt.close()

  0%|          | 0/100 [00:00<?, ?it/s]

In [52]:
with imageio.get_writer('figure/output.gif', mode='I', duration = 0.5) as writer:
    for filename in sorted(glob.glob("figure/output*.png")):
        image = imageio.imread(filename)
        writer.append_data(image)

/tmp/ipykernel_172162/3157470504.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
